In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import all the required libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import os
import glob as gb
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout
from zipfile import ZipFile
import matplotlib.image as mpimg
import warnings


sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

## Prepare data

In [ ]:
ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r").extractall()
ZipFile("/kaggle/input/dogs-vs-cats/test1.zip", "r").extractall()

In [ ]:
train_path = "./train"
test_path = "./test1"

In [ ]:
# Total number of images in train and test datasets
print('Total images in Train and Test Data Set:')
print(len(os.listdir(train_path)))
print(len(os.listdir(test_path)))


In [ ]:
filenames = os.listdir(train_path)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)

In [ ]:
data = pd.DataFrame({
    'Image': filenames,
    'Category': categories})
data.head(10)


In [ ]:
data['Category'].value_counts().plot.bar()

In [ ]:
#Split Train and Test Set
data_train, data_validation = train_test_split(data, test_size=0.1, random_state=18)
#(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
data_train = data_train.reset_index(drop=True)
data_validation = data_validation.reset_index(drop=True)

In [ ]:
data_train['Category'].value_counts().plot.bar()

In [ ]:
data_validation['Category'].value_counts().plot.bar()
#print(data_validation.loc[data_validation['Category'] == 'cat'].value_counts())
#print(data_validation.loc[data_validation['Category'] == 'dog'].value_counts())



In [ ]:
sample = random.choice(data['Image'])
plt.imshow(plt.imread(("./train/"+sample)))

## Setup Variables

In [ ]:
size  = 150
channels = 3
batch = 128
epochs = 50

## Data Augmentation
Let's make augmentation on training data and only scaling validation and test data.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             zoom_range=0.2,
                             width_shift_range=.2, height_shift_range=.2,
                             rotation_range=30,
                             brightness_range=[0.8, 1.2],
                             horizontal_flip=True)

In [ ]:
datagenValidation = ImageDataGenerator(rescale=1./255)

In [ ]:
X_train = datagen.flow_from_dataframe(
    data_train, 
    directory = train_path, 
    x_col = 'Image',
    y_col = 'Category',
    target_size= (size,size),
    class_mode = 'binary',
    shuffle = True,
    batch_size = batch)

In [ ]:
X_validation = datagenValidation.flow_from_dataframe(
    data_validation, 
    directory = train_path, 
    x_col = 'Image',
    y_col = 'Category',
    target_size= (size,size),
    class_mode = 'binary',
    shuffle = True,
    batch_size = batch)

In [ ]:
plt.figure(figsize = (12,12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in X_train:
        image = X_batch[0] #Y_batch is Category
        plt.imshow(image)
        break
        
plt.show()        


## Setup Callbacks

In [ ]:
checkpoint_filepath = 'best_model.hdf5'
callback_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)
callback_learningrate = ReduceLROnPlateau(monitor='val_accuracy', mode='max', min_delta=0.03, patience=3, factor=.5, min_lr=0.00001, verbose=1)
callbacks = [callback_checkpoint, callback_learningrate]

## Design the Neural Network

In [ ]:
Model = Sequential([Conv2D(filters=32,  kernel_size=(3,3), activation="relu", input_shape=(size,size,channels)),
                    BatchNormalization(),
                    MaxPool2D(2,2),
                    Dropout(0.2),
                 
                    Conv2D(filters=64,  kernel_size=(5,5), activation="relu"),
                    BatchNormalization(),
                    MaxPool2D(3,3),
                    Dropout(0.2),
                
                    Conv2D(filters=128, kernel_size=(7,7), activation="relu"),
                    BatchNormalization(),
                    MaxPool2D(4,4),
                    Dropout(0.2),
                 
                    Flatten(),
                    
                    Dense(units=4096, activation="relu"),                 
                    BatchNormalization(),
                    Dropout(0.4),
                    
                    Dense(units=1024, activation="relu"),
                    BatchNormalization(),
                    Dropout(0.3),
                    
                                        
                    Dense(units=2, activation="relu"),
                    BatchNormalization(),
                    Dropout(0.5),
                    
                    Dense(units=1, activation="sigmoid"),
])


Model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

Model.summary()

## Let's train

In [ ]:
history = Model.fit(X_train, validation_data=X_validation, epochs=epochs, callbacks=callbacks)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Let's predict

In [ ]:
Model = tf.keras.models.load_model('best_model.hdf5')


In [ ]:
filenames = os.listdir(test_path)
df_test = pd.DataFrame({'Image':filenames})
df_test.head(10)

In [ ]:
X_test = datagenValidation.flow_from_dataframe(
    df_test, 
    directory=test_path, 
    x_col='Image',
    y_col=None,
    class_mode=None,
    target_size= (size,size),
    batch_size=batch,
    shuffle=False
)

In [ ]:
X_train.class_indices

In [ ]:
y_pred = Model.predict(X_test)


In [ ]:

y_pred = np.where(y_pred > .5, "Dog", "Cat")
y_pred


In [ ]:
plt.figure(figsize=(15,15))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[0][i])
    plt.xlabel(y_pred[i])
    
plt.show()

In [ ]:
label = np.where(y_pred=="Cat", 0, 1)
label

## Let's submit

In [ ]:
df_submission = pd.read_csv('../input/dogs-vs-cats/sampleSubmission.csv')
df_submission['label'] = label
df_submission.head(10)

In [ ]:
df_submission.to_csv('submission.csv', index=False)

In [ ]:
submission = pd.read_csv('./submission.csv')
submission.head(10)